In [2]:
library(ADNIMERGE)
data(adnimerge)

Loading required package: Hmisc

Loading required package: lattice

Loading required package: survival

Loading required package: Formula

Loading required package: ggplot2


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:base’:

    format.pval, units




In [3]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.2 ──
✔ tibble  3.2.1      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
✔ purrr   0.3.5      
── Conflicts ────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::src()       masks Hmisc::src()
✖ dplyr::summarize() masks Hmisc::summarize()


In [ ]:
adnimerge %>%
  group_by(PTID) %>%
  filter( n_distinct(DX.bl) > 1 ) 

In [6]:
EMCI_Dementia = adnimerge %>%
  group_by(PTID) %>%
  filter(DX.bl == "EMCI" & DX == "Dementia") 

In [9]:
LMCI_Dementia = adnimerge %>%
  group_by(PTID) %>%
  filter(DX.bl == "LMCI" & DX == "Dementia") 

In [7]:
EMCI_Dementia_PITD = unique(EMCI_Dementia$PTID)

In [10]:
LMCI_Dementia_PITD = unique(LMCI_Dementia$PTID)

In [38]:
EMCI_1_PTID = read.csv("extdata/EMCI_1_names.csv")
EMCI_2_PTID = read.csv("extdata/EMCI_2_names.csv")


In [39]:
LMCI_1_PTID = read.csv("extdata/LMCI_1_names.csv")
LMCI_2_PTID = read.csv("extdata/LMCI_2_names.csv")


In [37]:
#EMCI_1_PTID_Dementia = intersect(EMCI_Dementia_PITD, EMCI_1_PTID$x)

In [16]:
#EMCI_2_PTID_Dementia = intersect(EMCI_Dementia_PITD, EMCI_2_PTID$x)

In [23]:
#LMCI_1_PTID_Dementia = intersect(LMCI_Dementia_PITD, LMCI_1_PTID$x)
#LMCI_2_PTID_Dementia = intersect(LMCI_Dementia_PITD, LMCI_2_PTID$x)

In [18]:
# get the modified df for the patient
get_modified_dementia_df <- function(i, adnimerge, PITD){
   adnimerge_MCI_Dementia = adnimerge[adnimerge$PTID == PITD[i], c("PTID","EXAMDATE", "DX.bl","DX")]

   # Fill the NA value
   adnimerge_MCI_Dementia = adnimerge_MCI_Dementia %>% 
       fill(DX, .direction = "updown")
   # print(adnimerge_MCI_Dementia)

   # Number of half year time intervals
   first_time = adnimerge_MCI_Dementia[1,"EXAMDATE"]
   last_time = adnimerge_MCI_Dementia[ length(row.names(adnimerge_MCI_Dementia)),"EXAMDATE"]
   diff = last_time - first_time
   time_interval  = as.integer(diff/182.5) + 1

   # Create modified df (tracking to the closest time before the timestamp)

   modified_df <- data.frame(matrix(ncol = 3, nrow = time_interval))
   colnames(modified_df) = c("PTID", "timestamp", 'DX')

   #k: index for adnimerge_MCI_Dementia
   k = 1
   current_time = adnimerge_MCI_Dementia[1, "EXAMDATE"]
   for (j in 1: time_interval){
    
       if(as.numeric(current_time - adnimerge_MCI_Dementia[k, "EXAMDATE"]) < 182.5){
           current_DX = adnimerge_MCI_Dementia[k, 'DX']
       }
       else{
           k = k + 1
           current_DX = adnimerge_MCI_Dementia[k , 'DX']
       } 
       #print(k)
       #print(current_DX)
       #print(current_time)
       modified_df[j, ] = c(adnimerge_MCI_Dementia[k, "PTID"], j , as.character(current_DX))
       current_time = current_time + 182.5
   }
   modified_df$timestamp = as.integer(modified_df$timestamp)
   return(modified_df)
}


In [51]:
# get the AD prop for a list

get_ad_prop <- function(adnimerge, PTID){
    merged_df = get_modified_dementia_df(1, adnimerge, PTID)[,c(-1)]

    for(i in 2:length(PTID)){
        df2 = get_modified_dementia_df(i, adnimerge, PTID)[,c(-1)]
        merged_df = merge(merged_df, df2, by = "timestamp", all.x = TRUE, all.y = TRUE)
    }
    rownames(merged_df) = merged_df$timestamp
    merged_df = merged_df[,c(-1)]
    merged_df[is.na(merged_df)] <- "No record"
    prop_list= list()
    for(i in 1:nrow(merged_df))
    {
        sum = 0
        for(j in 1: ncol(merged_df))
        {
            
            if(merged_df[i , j] == "MCI")
            {
                sum = sum + 1
            }
        }
        sum_dimentia = 0
         for(j in 1: ncol(merged_df))
        {
            
            if(merged_df[i , j] == "Dementia")
            {
                sum_dimentia = sum_dimentia + 1
            }
        }
        prop = sum/(sum+ sum_dimentia)
        prop_list[i] = 1 - prop
    }
    return(prop_list)
    
}

In [52]:
prop_EMCI_1 = suppressWarnings(get_ad_prop(adnimerge, EMCI_1_PTID$x))

In [ ]:
prop_EMCI_1

In [53]:
prop_EMCI_2 = suppressWarnings(get_ad_prop(adnimerge, EMCI_2_PTID$x))

In [56]:
prop_LMCI_1 = suppressWarnings(get_ad_prop(adnimerge, LMCI_1_PTID$x))

In [57]:
prop_LMCI_2 = suppressWarnings(get_ad_prop(adnimerge, LMCI_2_PTID$x))

In [60]:
write.csv(prop_EMCI_1, "extdata/prop_EMCI_1_all.csv")

In [61]:
write.csv(prop_EMCI_2, "extdata/prop_EMCI_2_all.csv")

In [62]:
write.csv(prop_LMCI_1, "extdata/prop_LMCI_1_all.csv")

In [63]:
write.csv(prop_LMCI_2, "extdata/prop_LMCI_2_all.csv")